In [8]:
import pandas as pd
import plotly.express as px

import dash
import dash_core_components as dcc
from dash import html
import dash_html_components as html
from dash.dependencies import Input, Output

In [9]:
app = dash.Dash(__name__)

In [10]:
df = pd.read_csv("data_moods.csv")
#df

,name,album,artist,id,release_date,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,mood
0,1999,1999,Prince,2H7PHVdQ3mXqEHXcvclTB0,1982-10-27,68,379266,0.866,0.13700,0.7300,0.000000,0.0843,0.6250,-8.201,0.0767,118.523,5,4,Happy
1,23,23,Blonde Redhead,4HIwL9ii9CcXpTOTzMq0MP,2007-04-16,43,318800,0.381,0.01890,0.8320,0.196000,0.1530,0.1660,-5.069,0.0492,120.255,8,4,Sad
2,9 Crimes,9,Damien Rice,5GZEeowhvSieFDiR8fQ2im,2006-11-06,60,217946,0.346,0.91300,0.1390,0.000077,0.0934,0.1160,-15.326,0.0321,136.168,0,4,Sad
3,99 Luftballons,99 Luftballons,Nena,6HA97v4wEGQ5TUClRM0XLc,1984-08-21,2,233000,0.466,0.08900,0.4380,0.000006,0.1130,0.5870,-12.858,0.0608,193.100,4,4,Happy
4,A Boy Brushed Red Living In Black And White,They're Only Chasing Safety,Underoath,47IWLfIKOKhFnz1FUEUIkE,2004-01-01,60,268000,0.419,0.00171,0.9320,0.000000,0.1370,0.4450,-3.604,0.1060,169.881,1,4,Energetic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,windcatcher,windcatcher,Leo Nocta,59VApBbrS2IADQk4ml5mdo,2020-06-19,36,123066,0.402,0.96100,0.2360,0.919000,0.0921,0.1460,-20.615,0.0603,129.736,0,3,Calm
682,yellow is the color of her eyes,yellow is the color of her eyes,Soccer Mommy,4D3nttJPU6L0M2epr7sId6,2019-11-19,5,435080,0.452,0.75700,0.5150,0.120000,0.1400,0.1910,-7.351,0.0255,80.537,11,4,Sad
683,you broke me first,you broke me first,Tate McRae,45bE4HXI0AwGZXfZtMp8JR,2020-04-17,87,169265,0.642,0.78600,0.3740,0.000000,0.0906,0.0799,-9.386,0.0545,124.099,4,4,Sad
684,you were good to me,brent,Jeremy Zucker,4CxFN5zON70B3VOPBYbd6P,2019-05-03,76,219146,0.561,0.91300,0.0848,0.000026,0.1120,0.2060,-15.099,0.0404,102.128,2,4,Sad


In [16]:
#vorbereiten der Daten
#Datensatz nach Release Date aufsteigend sortieren und neu indexieren
dfsort = df.sort_values(by=["release_date"])
dfsort = dfsort.reset_index(drop=True)
dfsort

#Um den Index nutzen zu können erstellen wir eine Spalte aus dem Index
dfsort['index1'] = dfsort.index
dfsort

#Ursprungsdatensatz für eine weitere Transformation kopieren und Datentyp der Spalte "mood" auf String stellen
dfcolmap = df
dfcolmap= dfcolmap.astype({"mood": str})
#Moods durch Zahlen ersetzen
dfcolmap["mood"].replace({"Happy": 1, "Sad": 2, "Energetic": 3, "Calm": 4}, inplace=True)
dfcolmap

#Ursprungsdatensatz kopieren
dfMoodspYear = df

#Release Datum aufspalten in Jahr, Monat und Tag
dfMoodspYear[["year", "month", "day"]] = dfMoodspYear["release_date"].str.split("-", expand=True)
dfMoodspYear
#Von den neuen Spalten behalten wir nur das Jahr
dfMoodspYear = dfMoodspYear.drop(columns=['month', 'day'])
dfMoodspYear
#Gruppieren und zählen nach Jahr und Mood
dfMoodspYear = df.groupby(["year", "mood"]).count()
dfMoodspYear
#Jahr und Mood wieder nutzbar machen und neuen Index setzen
MpY = dfMoodspYear.iloc[:,0]
MpY = MpY.reset_index(level=(0,1))
MpY["year"] = MpY["year"].astype("int")
MpY = MpY.sort_values(by=["year"])
MpY

,year,mood,name
0,1963,Happy,1
1,1966,Happy,1
2,1967,Happy,1
3,1969,Happy,3
4,1972,Happy,4
...,...,...,...
97,2019,Energetic,27
101,2020,Energetic,1
102,2020,Happy,2
100,2020,Calm,114


In [21]:
app.layout = html.Div([
    
        
    html.H1("Music Moods", style = {'text-align': 'center'}),
    html.H4("Vorhandenen Moods", style = {'text-align': 'left'}),

    dcc.Dropdown(id='mood',
                 options = [
                     {"label": "Happy", "value": 'Happy'},
                     {"label": "Calm", "value": 'Calm'},
                     {"label": "Sad", "value": 'Sad'},
                     {"label": "Energetic", "value": 'Energetic'}],
                 multi = False,
                 value='Happy',
                 style = {"width": "40%"}),
    
        
    dcc.Graph(id='mood_vergleich, figure = {}),
    html.Br(),
    
    dcc.Graph(id='mood_py', figure = {})
])


In [23]:
@app.callback(
    [Output(component_id='mood_vergleich', component_property='figure'),
     Output(component_id='mood_py', component_property='figure')],
    [Input(component_id='mood', component_property='value')]
)


SyntaxError: unexpected EOF while parsing (Temp/ipykernel_17136/679506191.py, line 5)

In [24]:
def update_graph(option_slctd):
    dff = df.copy()
    dff = dff[dff["mood"] == option_slctd]
    

In [28]:
# Plotly Express
    fig5 = px.bar(MpY, x="year", y="name", color="mood")

    dfcolmap = df
    dfcolmap["mood"].replace({"Happy": 1, "Sad": 2, "Energetic": 3, "Calm": 4}, inplace=True)
    fig3 = px.parallel_coordinates(dfcolmap, color="mood", color_continuous_scale=[(0.00, "red"), (0.25, "red"),
                                                                               (0.25, "yellow"), (0.50, "yellow"),
                                                                               (0.50, "aqua"),  (0.75, "aqua"),
                                                                               (0.75, "fuchsia"), (1.00, "fuchsia")]) 
    return fig5, fig3

IndentationError: unexpected indent (Temp/ipykernel_17136/273087454.py, line 2)

In [26]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Oct/2022 16:51:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2022 16:51:34] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2022 16:51:34] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2022 16:51:34] "GET /_favicon.ico?v=2.6.2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2022 16:51:34] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2022 16:51:34] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2022 16:51:34] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
